In [1]:
import sys, os, math
sys.path.insert(0, '..')
sys.path.insert(0, '../../quizx/pybindings')
import pyzx as zx
import quizx as qzx
import numpy as np
from pyzx.rewrite_rules.basicrules import *

In [2]:
c = zx.qasm("""
qreg q[4];
cx q[0], q[1];
cx q[2], q[1];
cx q[2], q[3];
cx q[3], q[0];
cx q[3], q[1];
cx q[2], q[1];
cx q[1], q[2];
""")
g = c.to_graph()
zx.draw(g, labels=True)

In [3]:
fuse(g, 4, 6)

True

In [4]:
g = c.to_graph()
while any(fuse(g, g.edge_s(e), g.edge_t(e)) for e in g.edges()): pass
zx.draw(g)

In [5]:
def finished(g, v):
    return ((g.type(v) == zx.VertexType.X) and any(w in g.outputs() for w in g.neighbors(v)) or
            (g.type(v) == zx.VertexType.Z) and any(w in g.inputs()  for w in g.neighbors(v)))

def strong_comp_safe(g, v0, v1):
    if finished(g, v0) or finished(g, v1): return False
    else: return strong_comp(g, v0, v1)

def phase_free_simp(g):
    while (
        any(fuse(g, g.edge_s(e), g.edge_t(e)) for e in g.edges()) or
        any(strong_comp_safe(g, g.edge_s(e), g.edge_t(e)) for e in g.edges())
    ): pass

In [6]:
g = c.to_graph()
phase_free_simp(g)
zx.draw(g)

In [7]:
c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=5, depth=400, clifford=True)
g = c.to_graph()

In [8]:
zx.draw(g)

In [9]:
zx.simplify.clifford_simp(g)
g.normalize()
zx.draw(g)

In [10]:
c1 = zx.extract_circuit(g.copy(), up_to_perm=True)
zx.draw(c1)

In [11]:
g = c.to_graph(backend="quizx-vec")

In [12]:
zx.draw(g)

In [13]:
gs = g.copy()
%time zx.simplify.clifford_simp(gs)
gs.normalize()
zx.draw(gs)

CPU times: user 183 ms, sys: 0 ns, total: 183 ms
Wall time: 182 ms


In [14]:
gs = g.copy()
%time qzx.simplify.clifford_simp(gs)
gs.normalize()
zx.draw(gs)

CPU times: user 1.32 ms, sys: 406 µs, total: 1.73 ms
Wall time: 1.65 ms
